Imports

In [46]:
import astropy.units as u
import astropy.coordinates as coord
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import numpy as np
import gala.integrate as gi
import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic
import scipy.optimize as opt
from scipy.optimize import minimize_scalar

Helper functions

In [47]:
def compare(x_vals1, y_vals1, x_vals2, y_vals2):
    pos1 = np.column_stack((x_vals1, y_vals1))
    mean1 = np.mean(pos1, axis=0)
    cov1 = np.cov(pos1, rowvar=False)

    pos2 = np.column_stack((x_vals2, y_vals2))
    mean2 = np.mean(pos2, axis=0)
    cov2 = np.cov(pos2, rowvar=False)

    dx = mean1[0] - mean2[0]
    dy = mean1[1] - mean2[1]

    combined_sigma_x = np.sqrt(np.max([cov1[0, 0],cov2[0, 0]]))
    combined_sigma_y = np.sqrt(np.max([cov1[1, 1],cov2[1, 1]]))

    Dx = dx / combined_sigma_x
    Dy = dy / combined_sigma_y

    return (Dx, Dy)

def final_points(dict, ics, num_steps):
    orbit = pot.integrate_orbit(ics, dt=-0.1*u.Myr, n_steps=num_steps)

    n_samples = 100
    ra_samples = np.random.normal(dict['ra'], dict['e_ra'], n_samples)
    dec_samples = np.random.normal(dict['dec'], dict['e_dec'], n_samples)
    dist_samples = np.random.normal(dict['dist'], dict['e_dist'], n_samples)
    pmra_samples = np.random.normal(dict['pmra'], dict['e_pmra'], n_samples)
    pmdec_samples = np.random.normal(dict['pmdec'], dict['e_pmdec'], n_samples)
    rv_samples = np.random.normal(dict['rv'], dict['e_rv'], n_samples)

    ics_list = []
    for ra, dec, dist, pmra, pmdec, rv in zip(ra_samples, dec_samples, dist_samples, pmra_samples, pmdec_samples, rv_samples):
        sc = coord.SkyCoord(ra=ra * u.degree,
                    dec=dec * u.degree,
                    distance=dist * u.kpc,
                    pm_ra_cosdec=pmra * (u.mas / u.yr),
                    pm_dec=pmdec * (u.mas / u.yr),
                    radial_velocity=rv * (u.km / u.s),
                    frame="icrs")
        gc = sc.transform_to(coord.Galactocentric)
        ics = gd.PhaseSpacePosition(pos=gc.cartesian.xyz, vel=gc.velocity.d_xyz)
        ics_list.append(ics)

    orbits = []
    orbits.append(orbit)
    for ics in ics_list:
        orbit = pot.integrate_orbit(ics, dt=-0.1*u.Myr, n_steps=num_steps)
        orbits.append(orbit)

    points = []
    for i, orbit in enumerate(orbits):
        final_pos = orbit[-1].pos.xyz.to(u.kpc).value[:3]
        points.append(final_pos)
    points = np.array(points)

    x = points[:, 0]
    y = points[:, 1]
    z = points[:, 2]

    return x, y, z

def calculate_distance(num_steps, star, dwarf_dict, dwarf_ics):
    SkyCoord = coord.SkyCoord(
        ra=star['ra'] * u.degree,
        dec=star['dec'] * u.degree,
        distance=star['dist'] * u.kpc,
        pm_ra_cosdec=star['pmra'] * (u.mas / u.yr),
        pm_dec=star['pmdec'] * (u.mas / u.yr),
        radial_velocity=star['rv'] * (u.km / u.s),
        frame="icrs"
    )
    GC = SkyCoord.transform_to(coord.Galactocentric)
    ics = gd.PhaseSpacePosition(pos=GC.cartesian.xyz, vel=GC.velocity.d_xyz)

    dx, dy, dz = final_points(dwarf_dict, dwarf_ics, num_steps)
    x, y, z = final_points(star, ics, num_steps)

    XY = compare(dx, dy, x, y)
    distance_xy = np.sqrt(XY[0]**2 + XY[1]**2)
    XZ = compare(dx, dz, x, z)
    distance_xz = np.sqrt(XZ[0]**2 + XZ[1]**2)
    YZ = compare(dy, dz, y, z)
    distance_yz = np.sqrt(YZ[0]**2 + YZ[1]**2)
    total = np.sqrt(distance_xy**2 + distance_xz**2 + distance_yz**2)

    return total

Define potential

In [48]:
pot = gp.MilkyWayPotential()

Stars

In [49]:
hvs5 = {
    'ra': 139.4978105603, #degree
    'e_ra': 0.0001, #degree
    'dec': 67.3773227272, #degree
    'e_dec': 0.0001, #degree
    'dist': 44.20, #kpc
    'e_dist': 5.09, #kpc
    'pmra': 0.00, #mas/yr
    'e_pmra': 0.08, #mas/yr
    'pmdec': -0.99, #mas/yr
    'e_pmdec': 0.11, #mas/yr
    'rv': 545.50, #km/s
    'e_rv': 4.30, #km/s
    }
hvs8 = {
    'ra': 145.5584849227, #degree
    'e_ra': 0.0001, #degree
    'dec': 20.0561234065, #degree
    'e_dec': 0.0001, #degree
    'dist': 53.19, #kpc
    'e_dist': 9.80, #kpc
    'pmra': -0.88, #mas/yr
    'e_pmra': 0.16, #mas/yr
    'pmdec': -0.28, #mas/yr
    'e_pmdec': 0.14, #mas/yr
    'rv': 499.30, #km/s
    'e_rv': 2.90, #km/s
    }
hvs14 = {
    'ra': 161.0072805184, #degree
    'e_ra': 0.0001, #degree
    'dec': 6.1941762509, #degree
    'e_dec': 0.0001, #degree
    'dist': 102.66, #kpc
    'e_dist': 16.55, #kpc
    'pmra': -2.17, #mas/yr
    'e_pmra': 1.38, #mas/yr
    'pmdec': 2.28, #mas/yr
    'e_pmdec': 1.68, #mas/yr
    'rv': 537.30, #km/s
    'e_rv': 7.20, #km/s
    }
hvs17 = {
    'ra': 250.4849449351, #degree
    'e_ra': 0.0001, #degree
    'dec': 47.3961264077, #degree
    'e_dec': 0.0001, #degree
    'dist': 49.82, #kpc
    'e_dist': 3.90, #kpc
    'pmra': -1.13, #mas/yr
    'e_pmra': 0.09, #mas/yr
    'pmdec': -0.93, #mas/yr
    'e_pmdec': 0.10, #mas/yr
    'rv': 250.20, #km/s
    'e_rv': 2.90, #km/s
    }
hvs23 = {
    'ra': 329.1209186025, #degree
    'e_ra': 0.0001, #degree
    'dec': 0.9122807743, #degree
    'e_dec': 0.0001, #degree
    'dist': 114.87, #kpc
    'e_dist': 20.10, #kpc
    'pmra': -1.21, #mas/yr
    'e_pmra': 1.29, #mas/yr
    'pmdec': -2.46, #mas/yr
    'e_pmdec': 1.50, #mas/yr
    'rv': 259.30, #km/s
    'e_rv': 9.80, #km/s
    }

map = {
    0: '5',
    1: '8',
    2: '14',
    3: '17',
    4: '23',
}

Dwarf galaxy

In [50]:
# print("Leo I")
# dwarf_dict = {
#     'ra': 152.117175, #degree
#     'e_ra': 0.0001, #degree
#     'dec': 12.3065, #degree
#     'e_dec': 0.0001, #degree
#     'dist': 254.0, #kpc
#     'e_dist': 15.5, #kpc
#     'pmra': -0.007, #mas/yr
#     'e_pmra': 0.035, #mas/yr
#     'pmdec': -0.119, #mas/yr
#     'e_pmdec': 0.026, #mas/yr
#     'rv': 282.9, #km/s
#     'e_rv': 0.5, #km/s
#     }
# print("Sextans")
# dwarf_dict = {
#     'ra': 153.262584, #degree
#     'e_ra': 0.0001, #degree
#     'dec': -1.6147, #degree
#     'e_dec': 0.0001, #degree
#     'dist': 95.0, #kpc
#     'e_dist': 3.0, #kpc
#     'pmra': -0.41, #mas/yr
#     'e_pmra': 0.01, #mas/yr
#     'pmdec': 0.04, #mas/yr
#     'e_pmdec': 0.01, #mas/yr
#     'rv': 224.3, #km/s
#     'e_rv': 0.1, #km/s
#     }
# print("Ursa Major I")
# dwarf_dict = {
#     'ra': 160, #degree
#     'e_ra': 0.0001, #degree
#     'dec': 56, #degree
#     'e_dec': 0.0001, #degree
#     'dist': 97.3, #kpc
#     'e_dist': 5.8, #kpc
#     'pmra': -0.39, #mas/yr
#     'e_pmra': 0.03, #mas/yr
#     'pmdec': -0.63, #mas/yr
#     'e_pmdec': 0.03, #mas/yr
#     'rv': -55.3, #km/s
#     'e_rv': 1.4, #km/s
#     }
# print("Willman I")
# dwarf_dict = {
#     'ra': 162.343, #degree
#     'e_ra': 0.0001, #degree
#     'dec': 51.051, #degree
#     'e_dec': 0.0001, #degree
#     'dist': 45.0, #kpc
#     'e_dist': 10.0, #kpc
#     'pmra': 0.21, #mas/yr
#     'e_pmra': 0.06, #mas/yr
#     'pmdec': -1.08, #mas/yr
#     'e_pmdec': 0.09, #mas/yr
#     'rv': -14.1, #km/s
#     'e_rv': 1.0, #km/s
#     }
# print("Leo II")
# dwarf_dict = {
#     'ra': 165, #degree
#     'e_ra': 0.0001, #degree
#     'dec': 22.15, #degree
#     'e_dec': 0.0001, #degree
#     'dist': 233.0, #kpc
#     'e_dist': 14.0, #kpc
#     'pmra': -0.14, #mas/yr
#     'e_pmra': 0.02, #mas/yr
#     'pmdec': -0.12, #mas/yr
#     'e_pmdec': 0.02, #mas/yr
#     'rv': 78.3, #km/s
#     'e_rv': 0.6, #km/s
#     }
# print("Leo V")
# dwarf_dict = {
#     'ra': 172.79, #degree
#     'e_ra': 0.0001, #degree
#     'dec': 2.22, #degree
#     'e_dec': 0.0001, #degree
#     'dist': 169.0, #kpc
#     'e_dist': 4.0, #kpc
#     'pmra': -0.06, #mas/yr
#     'e_pmra': 0.09, #mas/yr
#     'pmdec': -0.25, #mas/yr
#     'e_pmdec': 0.085, #mas/yr
#     'rv': 170.9, #km/s
#     'e_rv': 2.0, #km/s
#     }
print("Leo IV")
dwarf_dict = {
    'ra': 173.2375, #degree
    'e_ra': 0.0001, #degree
    'dec': -0.5333, #degree
    'e_dec': 0.0001, #degree
    'dist': 154.0, #kpc
    'e_dist': 5.0, #kpc
    'pmra': -0.08, #mas/yr
    'e_pmra': 0.09, #mas/yr
    'pmdec': -0.21, #mas/yr
    'e_pmdec': 0.08, #mas/yr
    'rv': 132.3, #km/s
    'e_rv': 1.4, #km/s
    }
dwarf = coord.SkyCoord(
    ra=dwarf_dict['ra'] * u.degree,
    dec=dwarf_dict['dec'] * u.degree,
    distance=dwarf_dict['dist'] * u.kpc,
    pm_ra_cosdec=dwarf_dict['pmra'] * (u.mas / u.yr),
    pm_dec=dwarf_dict['pmdec'] * (u.mas / u.yr),
    radial_velocity=dwarf_dict['rv'] * (u.km / u.s),
    frame="icrs"
)
dwarf_GC = dwarf.transform_to(coord.Galactocentric)
dwarf_ics = gd.PhaseSpacePosition(pos=dwarf_GC.cartesian.xyz, vel=dwarf_GC.velocity.d_xyz)


Leo IV


Calculate

In [51]:
for i, star in enumerate([hvs5, hvs8, hvs14, hvs17, hvs23]):
    result = minimize_scalar(
        calculate_distance,
        args=(star, dwarf_dict, dwarf_ics),
        bounds=(0, 3000),  # search between 0 and 3000 steps (300 Myr)
        method='bounded'
    )

    # print(f"hvs{map[i]}: minimum distance {result.fun}")
    print(f"min{map[i]} = {result.x}")

min5 = 7417.053338030337
min8 = 7639.320225002102
min14 = 6180.317197042747
min17 = 5976.493930990124
min23 = 4721.367758272425
